In [2]:
import os
import pandas as pd
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [3]:
df = pd.read_csv("./tweet_emotions.csv")

llm= ChatOpenAI(temperature=0.9)
df.head()

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_12992\338911499.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm= ChatOpenAI(temperature=0.9)


,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
prompt_template = ChatPromptTemplate.from_template(
    "What the best name to describe someone \
        that is feeling this way {feeling}?"
)

In [5]:
chain = LLMChain(llm=llm, prompt=prompt_template)

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_12992\1546045208.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [8]:
feeling = df.content[:3]

In [9]:
chain.run(feeling)

'The best name to describe someone feeling this way would be "sad" or "depressed."'

#### SimpleSequentialChain
Works best when you expect 1 input and 1 output


In [10]:
from langchain.chains import SimpleSequentialChain

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description to help someone  \
    feeling this way:{feelings_description}"
) 
chain_two = LLMChain(llm=llm, prompt= second_prompt)

In [11]:
overall_simple_chain = SimpleSequentialChain(chains=[chain,chain_two],
                                            verbose=True
                                             )

In [10]:
overall_simple_chain.run(feeling)



> Entering new SimpleSequentialChain chain...
"Anxious and Anticipatory"
Feeling anxious and anticipatory means you are on edge, constantly worrying about the future. Take deep breaths and stay present.

> Finished chain.


'Feeling anxious and anticipatory means you are on edge, constantly worrying about the future. Take deep breaths and stay present.'

Sequential Chain with morethan one input

In [12]:
from langchain.chains import SequentialChain

In [18]:

chain_one = LLMChain(llm=llm, prompt=prompt_template, 
                     output_key="feeling_description"
                    )


In [19]:
prompt_template_2 = ChatPromptTemplate.from_template(
    "Can you summarize the following feelings in 1 sentence:"
    "\n \n {feeling_description}"
)
chain_two = LLMChain(llm=llm, prompt=prompt_template_2, output_key="feeling_summary")

In [ ]:
prompt_template_3 = ChatPromptTemplate.from_template(
    "What the name of this feeling:\n\n{feeling_summary}"
)
chain_three = LLMChain(llm=llm, prompt=prompt_template_3, output_key="feeling_name")

In [21]:

prompt_template_4 = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary of the feeling:"
    "\n\n Summary: {feeling_name}\n\nLanguage: {feeling_name}"
)

chain_four = LLMChain(llm=llm, prompt=prompt_template_4,
                      output_key="followup_message"
                     )


In [24]:
sequential_chain = SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables = ["feeling"],
    output_variables = ["feeling_description","feeling_summary","followup_message" ],
    verbose=True
)


In [26]:
sequential_chain(feeling)

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_12992\782359686.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sequential_chain(feeling)




> Entering new SequentialChain chain...

> Finished chain.


{'feeling': 0    @tiffanylue i know  i was listenin to bad habi...
 1    Layin n bed with a headache  ughhhh...waitin o...
 2                  Funeral ceremony...gloomy friday...
 Name: content, dtype: object,
 'feeling_description': 'The best name to describe someone feeling this way would be "sad" or "gloomy."',
 'feeling_summary': 'Feeling downcast and disheartened.',
 'followup_message': "I'm sorry to hear that you're feeling dejected. It's completely normal to have moments of sadness or disappointment, but remember that these feelings are temporary and you will get through this. Allow yourself to feel your emotions and then try to focus on the positive aspects of your life. Remember that you are not alone and there are people who care about you. If you ever need someone to talk to, I'm here to listen. Stay strong."}